# 🌿 Global Spice Intelligence AI Assistant
## Introduction & Overview
Welcome to the world of Global Spice Intelligence — an AI-powered wellness assistant designed to bridge the gap between ancient wisdom and modern technology.

This project introduces a conversational AI that offers personalized guidance on the use of spices and natural remedies for health and beauty. By blending traditional healing systems with cutting-edge AI tools, it empowers users to explore natural wellness in a safe, intelligent, and accessible way.

### ✨ What Can This Assistant Do?
* **Understand Your Needs:**
It analyzes your input to detect health concerns, beauty goals, and even mentions of medications.

* **Recommend Natural Remedies:**
Based on systems like Ayurveda, Traditional Chinese Medicine, and other global traditions, it suggests herbs and spices that align with your needs.

* **Explain the Science:**
It shares evidence-based insights about the properties, benefits, and traditional uses of each recommended spice.

* **Pull From the Web (When Needed):**
If the internal database doesn’t have an answer, the assistant can tap into web-based sources to offer additional, up-to-date knowledge.

* **Put Safety First:**
While the assistant is smart, it always reminds you to consult with a medical professional for serious conditions or before making health decisions.

In [1]:
# Cell 1: Install Dependencies
# Install the google-genai SDK.
!pip install -qU 'google-genai==1.7.0'

In [2]:
# Cell 2: Import Libraries 
# Main Bot
import pandas as pd
import sqlite3
import spacy
import os

# Grounding
from google import genai
from google.genai import types

# Image and Document Understanding
from PIL import Image
import io

# Import Kaggle Seccret
from kaggle_secrets import UserSecretsClient
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
client = genai.Client(api_key=GOOGLE_API_KEY)

In [3]:
# Cell 3: List available Gemini models
for model in client.models.list():
    print(model.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
mod

### 🌞 Agents and Their Functions
To bring this intelligent wellness assistant to life, we’ve designed a system of specialized AI agents — each with a unique role. These agents collaborate to understand user input, retrieve relevant knowledge, and generate meaningful responses.

Let’s take a closer look at how each one contributes to the experience.

In [4]:
# Cell 4: Base Agent Class
class BaseAgent:
    """Base class for all agents"""
    def __init__(self, name):
        self.name = name

    def process(self, input_data):
        raise NotImplementedError("Subclasses must implement this method")

### 🧠 InputProcessingAgent
The `InputProcessingAgent` is the brain behind understanding what the user truly needs. Here's how it works:

* **Understands What You’re Asking:**
Whether you're mentioning a symptom, a prescription, or a skincare concern, this agent picks up on the key details.

* **Uses Smart NLP Techniques:**
By applying natural language processing (NLP), it extracts the essential information from your message—quickly and accurately.

* **Prepares Data for the Next Step:**
It transforms your input into structured data that other agents can use to give you helpful, personalized recommendations.

👉 Think of this as the first friend you talk to when you have a wellness question—it listens closely so the whole team can respond better.

### 📚 KnowledgeAgent
The `KnowledgeAgent` is your personal herbal researcher—it knows where to look and what to find. Here's what it does:

* **Finds the Right Spice for the Right Need:**
Whether you mention a symptom, a drug interaction, a specific spice, or even a beauty concern, it knows exactly how to search the database for the most relevant matches.

* **Brings Back Helpful, Science-Based Info:**
It doesn't just find spices—it tells you what they do, how they're used, and why they might help, all grounded in trusted sources.

👉 Think of this agent as the “wise elder” of the team—quietly powerful, with deep knowledge of natural wellness.

In [5]:
# Cell 9: Input Agent
class InputAgent:
    def analyze_text(self, text):
        """Analyze user input for symptoms, medications, and beauty needs."""
        analysis = {
            'symptoms': [],
            'drugs': [],
            'beauty_need': False,
            'raw_text': text,
        }

        text_lower = text.lower()

        # Symptom detection
        symptoms_mapping = {
            'headache': ['headache', 'head pain', 'migraine'],
            'stomach pain': ['stomach pain', 'stomachache', 'belly ache'],
            'dry skin': ['dry skin', 'skin dryness', 'flaky skin']
        }

        for symptom, keywords in symptoms_mapping.items():
            if any(keyword in text_lower for keyword in keywords):
                analysis['symptoms'].append(symptom)

        # Medication detection
        medications = ['aspirin', 'ibuprofen', 'paracetamol']
        for med in medications:
            if med in text_lower:
                analysis['drugs'].append(med)

        # Beauty need detection
        beauty_keywords = ['beauty', 'skin care', 'skin', 'face', 'hair', 'cosmetic']
        if any(keyword in text_lower for keyword in beauty_keywords):
            analysis['beauty_need'] = True


        return analysis

### RecommendationAgent

The `RecommendationAgent` generates spice and remedy recommendations based on the user's input and knowledge from the `KnowledgeAgent`. It considers:

*   Symptoms, medications, and beauty needs identified by the `InputProcessingAgent`.
*   Traditional healing practices and spice properties from the database.
*   Generates personalized recommendations for spices and remedies.

In [6]:
# Cell 10: Recommendation Agent
class RecommendationAgent:
    def generate_recommendations(self, analysis):
        """Generate spice and remedy recommendations based on analysis."""
        recommendations = {
            'spices': [],
            'remedies': [],
            'warnings': []
        }

        # Spice and remedy database
        recommendation_db = {
            'headache': {
                'spices': ['turmeric', 'ginger'],
                'remedies': ['ginger tea', 'peppermint oil massage']
            },
            'stomach pain': {
                'spices': ['fennel', 'peppermint'],
                'remedies': ['chamomile tea', 'warm lemon water']
            },
            'dry skin': {
                'spices': ['cinnamon', 'turmeric'],
                'remedies': ['honey face mask', 'coconut oil application']
            },
            'beauty': {
                'spices': ['turmeric', 'saffron'],
                'remedies': ['yogurt and honey mask', 'aloe vera gel']
            }
        }

        # Generate recommendations based on symptoms
        for symptom in analysis['symptoms']:
            if symptom in recommendation_db:
                recommendations['spices'].extend(recommendation_db[symptom]['spices'])
                recommendations['remedies'].extend(recommendation_db[symptom]['remedies'])

        # Add beauty recommendations if needed
        if analysis['beauty_need']:
            recommendations['spices'].extend(recommendation_db['beauty']['spices'])
            recommendations['remedies'].extend(recommendation_db['beauty']['remedies'])

        # Add warnings if medications are mentioned
        if analysis['drugs']:
            meds_list = ", ".join(analysis['drugs'])
            recommendations['warnings'].append(
                f"Consult a doctor before using spices with {meds_list}"
            )

        # Remove duplicates
        recommendations['spices'] = list(set(recommendations['spices']))
        recommendations['remedies'] = list(set(recommendations['remedies']))

        return recommendations

### 🤖 ResponseAgent
The `ResponseAgent` is the core component responsible for generating conversational, user-friendly replies. Here's what it does:

* Formats Recommendations Clearly
Turns raw outputs into easy-to-understand messages for everyday users.

* Adds Warnings and Disclaimers
Ensures the user is aware of any important health-related notices.

* Fallback with Gemini Grounding
If the system lacks data for a specific query, it intelligently searches trusted sources on the web via Gemini’s grounding to generate helpful suggestions.

👉 This module ensures that every reply feels more like a caring wellness coach than a rigid AI. Let it help guide your journey—naturally.

In [7]:
# Cell 11: Response Agent and the Grounding function
class ResponseAgent:
    def generate_response(self, recommendations, analysis):
        """Generate a user-friendly response from recommendations."""
        response = []

        if recommendations['spices']:
            response.append("🌿 Recommended spices: " + ", ".join(recommendations['spices']))

        if recommendations['remedies']:
            response.append("💡 Natural remedies: " + ", ".join(recommendations['remedies']))

        if recommendations['warnings']:
            response.append("⚠️ " + "\n⚠️ ".join(recommendations['warnings']))

        if not any(recommendations.values()): # Check if all recommendation lists are empty
            config_with_search = types.GenerateContentConfig(
                tools=[types.Tool(google_search=types.GoogleSearch())],
            )
            # ... (Same fallback_prompt as before) ...

            fallback_prompt = (
            f"You are a Global Traditional Medicine Suggestion Bot. You will Based on the user's query: '{analysis['raw_text']} to offer offer suggestions rooted in traditional healing systems "
            f"from around the world, including Ayurveda, Traditional Chinese Medicine, African Herbalism, Native American remedies, and others. "
            f"You can only discuss topics related to these healing traditions, their remedies, and the wisdom they offer in nutrition, beauty, and wellness. "
            f"Do not offer modern medical advice or make medical diagnoses. You are not a substitute for a healthcare provider, and you must not recommend anything outside the scope of traditional practices.\n\n"
            f"Based on this input, identify relevant symptoms, any mentioned herbs or therapies, and the cultural context if available. "
            f"Generate a culturally appropriate list of traditional remedies or practices that may be helpful. "
            f"These can include food-based remedies, breathing techniques, herbal concoctions, or wellness rituals, as long as they are rooted in traditional practice. "
            f"Remind the user that these are not medical treatments and always advise consulting certified traditional healers or health professionals as needed. "
            f"Do not try to continue the conversation after you finished giving a whole response. "
            )

            try:
                response_web = client.models.generate_content(
                    model='gemini-2.0-flash',
                    contents=fallback_prompt,
                    config=config_with_search,
                )
                rc = response_web.candidates[0]
                response.append(f"I couldn't find specific spice recommendations in my database, but here's something from the web:\n\n{rc.content.parts[0].text}")
            except Exception as e:
                response.append("I'm having trouble retrieving web-based information right now. Please try again later.")


        if not response:
            response.append("I recommend consulting a healthcare professional for personalized advice.")

        return "\n".join(response)

### 🧪 The Chatbox
Because Kaggle does not support raw input saving within the notebook, we’ve commented out the main chat interface code by default.

👉 To try it out:
Please scroll down to the code cell under **# Uncomment the whould chunk below to activate the chatbox** and uncomment it to activate the main functionality of our AI assistant.

This will allow you to enter your own questions and receive real-time personalized suggestions based on your health needs!

In [8]:
# Cell 12: Spice AI Assistant
# Run with text input mode
# Uncomment the whould chunk below to activate the chatbox

class SpiceAIAssistant:
    def __init__(self):
        self.input_agent = InputAgent()
        self.recommendation_agent = RecommendationAgent()
        self.response_agent = ResponseAgent()


if __name__ == "__main__":
    assistant = SpiceAIAssistant()
    print("Welcome to Global Spice Intelligence AI Assistant!")
    print("How can I help you with health or beauty needs today?")
    print("Examples:\n- 'I have a headache'\n- 'Natural remedies for dry skin'\n- 'What helps with stomach pain?'")

    while True:
        try:
            text = input("\n> ").strip()
            if text.lower() in ['exit', 'quit', 'stop', 'bye']:
                print("Goodbye! Stay healthy!")
                break
            if not text:
                print("Please describe your health or beauty concern")
                continue

            analysis = assistant.input_agent.analyze_text(text)

            print("\nAnalysis:")
            if analysis['symptoms']:
                print(f"- Detected symptoms: {', '.join(analysis['symptoms'])}")
            if analysis['drugs']:
                print(f"- Mentioned medications: {', '.join(analysis['drugs'])}")
            if analysis['beauty_need']:
                print("- Beauty-related request detected")

            recommendations = assistant.recommendation_agent.generate_recommendations(analysis)
            response = assistant.response_agent.generate_response(recommendations, analysis)

            print("\nAI Recommendation:")
            print(response)

        except KeyboardInterrupt:
            print("\nGoodbye! Hope I was helpful!")
            break
        except Exception as e:
            print(f"\nError: {e}\nLet's try again...")


Welcome to Global Spice Intelligence AI Assistant!
How can I help you with health or beauty needs today?
Examples:
- 'I have a headache'
- 'Natural remedies for dry skin'
- 'What helps with stomach pain?'



>  I have a headache



Analysis:
- Detected symptoms: headache

AI Recommendation:
🌿 Recommended spices: turmeric, ginger
💡 Natural remedies: peppermint oil massage, ginger tea



>  I have a dry skin



Analysis:
- Detected symptoms: dry skin
- Beauty-related request detected

AI Recommendation:
🌿 Recommended spices: cinnamon, saffron, turmeric
💡 Natural remedies: coconut oil application, yogurt and honey mask, honey face mask, aloe vera gel



>  I have a fever



Analysis:

AI Recommendation:
I couldn't find specific spice recommendations in my database, but here's something from the web:

Okay, I understand. You're experiencing a fever. Here are some traditional approaches to managing fever from various global healing systems. Please remember that these are traditional practices and not medical advice. Consult with a qualified healthcare professional for any health concerns.

**Traditional Chinese Medicine (TCM):**

*   **Cooling Foods:** In TCM, fever is often seen as an excess of heat in the body. Consume cooling foods like cucumbers, watermelon, and mung bean soup.
*   **Herbal Teas:** Chrysanthemum tea or peppermint tea can help to dispel heat.
*   **Acupressure:** Specific acupressure points, like LI-11 (Quchi) on the elbow, may help reduce fever. (Consult a trained TCM practitioner for proper point location and technique).

**Ayurveda:**

*   **Cooling Herbs:** Coriander, sandalwood, and fennel are considered cooling herbs in Ayurveda. 


>  exit


Goodbye! Stay healthy!


### 📄 Document Understanding
Powered by Google Gemini 2.0 Flash, the AI excels at interpreting complex documents that are rich in specialized jargon—making traditional knowledge more accessible and actionable for everyday users.

In [9]:
# Cell 13: Document Understanding and Analyzing
def analyze_text_document(document_path):
    """Process traditional medicine documents with correct model name"""
    try:
        with open(document_path, 'r', encoding='utf-8') as file:
            text = file.read()
    except UnicodeDecodeError:
        try:
            with open(document_path, 'r', encoding='latin-1') as file:
                text = file.read()
        except Exception as e:
            return f"Error reading file: {str(e)}"


    response = client.models.generate_content(
        model='gemini-2.0-flash',
        contents=[text],
        config=types.GenerateContentConfig(
            system_instruction=(

                "Extract key traditional medicine knowledge from this document, "
                "organizing by: tradition, remedy, use case, preparation. "

            ),
            temperature=0.3,
            max_output_tokens=2000
        )
    )

    return response.text
text_result=analyze_text_document("/kaggle/input/tcm-medicine/dong.txt")

print(text_result)

Here's a breakdown of the traditional medicine knowledge extracted from the document, organized as requested:

**Tradition:** Traditional Chinese Medicine (TCM)

**Remedy:** Dong Quai (Angelica sinensis) root

**Use Cases:**

*   **Menopausal Symptoms:**
    *   Hot flushes
    *   Night sweats
    *   Sleep quality improvement (in combination with other herbs)
*   **Premature Ejaculation:** (Topical application, in combination with other herbs)

**Preparation:**

*   **Oral:**
    *   Often used in combination products with other herbs.
    *   Dosages up to 150 mg daily have been used safely in combination products for up to 6 months.
*   **Topical:**
    *   Applied directly to the skin of the penis in combination with other herbs for premature ejaculation.

**Important Considerations:**

*   **Safety:** Dong quai is *possibly safe* when taken orally for up to 6 months in low doses. Higher doses or prolonged use may be unsafe.
*   **Side Effects:** May cause skin sensitivity to sunl

### 🎨 Image Understanding
Powered by Google Gemini 2.0 Flash, the AI delivers high-performance visual recognition — capable of accurately interpreting complex images of herbs, spices, and medicinal plants, even when visual details are subtle or varied.

In [ ]:
# Cell 14: Image Understanding and Analyzing
def analyze_medicine_image(image_path):
    """Process traditional medicine images with correct model name"""
    try:
        try:
            img = Image.open(image_path)
        except Exception as e:
            return f"Error opening image: {str(e)}"

        # Initialize the vision model

        response = client.models.generate_content(
            model='gemini-2.0-flash',
            contents=[img],
            config=types.GenerateContentConfig(
                system_instruction=(
                    "Analyze this image of traditional medicine and provide:\n"
                    "1. Identification of any visible plants/materials\n"
                    "2. Likely cultural origins/traditions\n"
                    "3. Common medicinal uses\n"
                    "4. Preparation methods if discernible\n"
                    "5. Any safety considerations\n"
                    "Format your response clearly with these headings"
                ),
                temperature=0.3,
                max_output_tokens=2000
            )
        )

        return response.text

    except Exception as e:
        return f"Error analyzing image: {str(e)}"

# Example usage:
img_result = analyze_medicine_image("/kaggle/input/tcm-medicine/images.jpeg")
print(img_result)

### Future Improvements
* **User-Friendly Front-End Interface:**
  Build a comprehensive front-end interface to move beyond the current notebook environment, making the platform more accessible and intuitive for everyday users.
* M**ultilingual Expansion:**
  Extend language support beyond English to include Hindi, Mandarin, French, Spanish, and more — enhancing accessibility across global regions and cultural backgrounds.
* **Advanced Generative AI Enhancements**
  Incorporate cutting-edge GenAI capabilities such as few-shot prompting, fine-tuning, and GenAI-based evaluation to boost response accuracy, personalization, and contextual relevance.